Let's start by setting up some simulation parameters for the market. We'll do calculations in WEI to keep close to the original code. We will convert to ETH as necessary. As a preliminary step, let's define some market constants

In [1]:
WEI = 1
GWEI = 10**9
WHOLE = 10**18

def wei_to_whole(amount):
    return amount // 10**18

def gwei_to_whole(amount):
    return amount // 10**9

def gwei_to_wei(amount):
    return amount * 10**9

def wei_to_gwei(amount):
    return amount // 10**9

def convert_investment_price(price):
    """Investment price is ETH wei / MKT gwei. Convert to ETH whole / MKT whole"""
    # ETH gwei / MKT gwei
    price_gwei_to_gwei = wei_to_gwei(price)
    # Note units cancel
    price_whole_to_whole = price_gwei_to_gwei
    return price_whole_to_whole

Here are the market parameters. The rest of the notebook is designed so that if you change these parameters, the rest of the notebook should execute correctly

In [2]:
PRICE_FLOOR = int(.001 * GWEI)
SPREAD = 110 # 10 %
LIST_REWARD = .000025 * WHOLE # MKT
STAKE = .01 * WHOLE # MKT
VOTE_BY = 7 # Days. Are units right?
BACKEND_PAYMENT = 25
MAKER_PAYMENT = 25
RESERVE_PAYMENT = 100 - BACKEND_PAYMENT - MAKER_PAYMENT
COST_PER_BYTE = 100 * GWEI # ETH

Ok, now let's define the parameters that govern the current simulation.

In [3]:
N_INVESTORS = 10
# Investment each investor makes
INVESTMENT = 100 * WHOLE # ETH
N_MAKERS = 1000
# This is number of listings *per* maker
N_LISTINGS_PER_MAKER = 100
# Size of an individual data purchase in ETH
PURCHASE_SIZE = 100 * WHOLE # ETH
# Number of purchases
N_PURCHASES = 90

Let's have the market creator create the initial data market.

In [4]:
RESERVE = 0 # Initial reserve is empty
MARKET_TOTAL = 0 # No Market tokens at start

# Creator does their thing
CREATOR_BLOCK_SIZE = 1 * WHOLE # MKT
MARKET_TOTAL += CREATOR_BLOCK_SIZE
# Creator deposits 100 ETH in reserve
RESERVE += 100 * WHOLE # ETH
print("MARKET_TOTAL: %d, RESERVE: %d" % (wei_to_whole(MARKET_TOTAL), wei_to_whole(RESERVE)))

MARKET_TOTAL: 1, RESERVE: 100


Let's introduce a data structure to keep track of the coin table.

In [5]:
coin_table = {}
# We're just going to use whole units for convenience
coin_table["CREATOR"] = CREATOR_BLOCK_SIZE / 10**18

Now some investors join the market.

In [6]:
print("INVESTMENT: %d" % wei_to_whole(INVESTMENT))
for i in range(N_INVESTORS):
    print("Investor %d investing" % (i+1))
    INVESTMENT_PRICE = PRICE_FLOOR + ((SPREAD * RESERVE * GWEI) // (100 * MARKET_TOTAL))
    print("INVESTMENT_PRICE (ETH whole for MKT whole): %d" % convert_investment_price(INVESTMENT_PRICE))
    MINTED = (INVESTMENT // INVESTMENT_PRICE) * GWEI # Units of WEI
    print("MINTED: %f" % (MINTED / 10**18))
    MARKET_TOTAL += MINTED
    RESERVE += INVESTMENT
    coin_table["INVESTOR_%d" % (i+1)] = MINTED / 10**18

print("############################")
print("MARKET_TOTAL: %f" % (MARKET_TOTAL / 10**18))
print("RESERVE: %f" % (RESERVE / 10**18))
print("coin_table")
print(coin_table)

INVESTMENT: 100
Investor 1 investing
INVESTMENT_PRICE (ETH whole for MKT whole): 110
MINTED: 0.909083
Investor 2 investing
INVESTMENT_PRICE (ETH whole for MKT whole): 115
MINTED: 0.867757
Investor 3 investing
INVESTMENT_PRICE (ETH whole for MKT whole): 118
MINTED: 0.841460
Investor 4 investing
INVESTMENT_PRICE (ETH whole for MKT whole): 121
MINTED: 0.822334
Investor 5 investing
INVESTMENT_PRICE (ETH whole for MKT whole): 123
MINTED: 0.807381
Investor 6 investing
INVESTMENT_PRICE (ETH whole for MKT whole): 125
MINTED: 0.795147
Investor 7 investing
INVESTMENT_PRICE (ETH whole for MKT whole): 127
MINTED: 0.784820
Investor 8 investing
INVESTMENT_PRICE (ETH whole for MKT whole): 128
MINTED: 0.775901
Investor 9 investing
INVESTMENT_PRICE (ETH whole for MKT whole): 130
MINTED: 0.768063
Investor 10 investing
INVESTMENT_PRICE (ETH whole for MKT whole): 131
MINTED: 0.761080
############################
MARKET_TOTAL: 9.133027
RESERVE: 1100.000000
coin_table
{'CREATOR': 1.0, 'INVESTOR_1': 0.909082

Now let's get some makers involved in the system.

In [7]:
for i in range(N_MAKERS):
    for j in range(N_LISTINGS_PER_MAKER):
        MAKER = "MAKER_%d" % (i+1)
        MINTED = LIST_REWARD
        MARKET_TOTAL += MINTED
        if MAKER not in coin_table:
            coin_table[MAKER] = 0
        coin_table[MAKER] += MINTED/10**18

print("#####################")
print("RESERVE: %f" % (RESERVE / 10**18))
print("coin_table")
print(coin_table)

#####################
RESERVE: 1100.000000
coin_table
{'CREATOR': 1.0, 'INVESTOR_1': 0.909082644, 'INVESTOR_2': 0.867757308, 'INVESTOR_3': 0.841459571, 'INVESTOR_4': 0.822334038, 'INVESTOR_5': 0.807381401, 'INVESTOR_6': 0.795147459, 'INVESTOR_7': 0.784820128, 'INVESTOR_8': 0.775901087, 'INVESTOR_9': 0.768063154, 'INVESTOR_10': 0.761080279, 'MAKER_1': 0.0024999999999999988, 'MAKER_2': 0.0024999999999999988, 'MAKER_3': 0.0024999999999999988, 'MAKER_4': 0.0024999999999999988, 'MAKER_5': 0.0024999999999999988, 'MAKER_6': 0.0024999999999999988, 'MAKER_7': 0.0024999999999999988, 'MAKER_8': 0.0024999999999999988, 'MAKER_9': 0.0024999999999999988, 'MAKER_10': 0.0024999999999999988, 'MAKER_11': 0.0024999999999999988, 'MAKER_12': 0.0024999999999999988, 'MAKER_13': 0.0024999999999999988, 'MAKER_14': 0.0024999999999999988, 'MAKER_15': 0.0024999999999999988, 'MAKER_16': 0.0024999999999999988, 'MAKER_17': 0.0024999999999999988, 'MAKER_18': 0.0024999999999999988, 'MAKER_19': 0.0024999999999999988, 'M

Ok now that we have investors and makers on the coin table, let's get some purchases made.

In [8]:
BACKEND_REVENUE = 0
for n in range(N_PURCHASES):
    print("Purchase %d" % n)
    PURCHASE = PURCHASE_SIZE
    BACKEND_SHARE = (PURCHASE * BACKEND_PAYMENT)//100
    MAKER_SHARE = (PURCHASE * MAKER_PAYMENT)//100
    RESERVE_SHARE = (PURCHASE * RESERVE_PAYMENT)//100
    # Pay out the reserve share
    RESERVE += RESERVE_SHARE
    # Backend share is paid directly to backend
    BACKEND_REVENUE += BACKEND_SHARE
    # The maker share is invested via the invest() function. Let's find the current investment price:
    INVESTMENT_PRICE = PRICE_FLOOR + ((SPREAD * RESERVE * GWEI) // (100 * MARKET_TOTAL))
    print("INVESTMENT_PRICE (ETH whole for MKT whole): %d" % convert_investment_price(INVESTMENT_PRICE))
    # Now mint the makers their access rewards
    ONE_MAKER_SHARE = MAKER_SHARE // N_MAKERS
    MINTED = (ONE_MAKER_SHARE // INVESTMENT_PRICE) * GWEI # Units of WEI
    print("MINTED per maker: %f" % (MINTED / 10**18))
    for i in range(N_MAKERS):
        MAKER = "MAKER_%d" % (i+1)
        coin_table[MAKER] += MINTED/10**18
    
print("#####################")
print("coin_table")
print(coin_table)
print("MARKET_TOTAL: %f" % (MARKET_TOTAL / 10**18))
print("RESERVE: %f" % (RESERVE / 10**18))
CREATOR_ETH = (coin_table["CREATOR"]/(MARKET_TOTAL / 10**18)) * (RESERVE / 10**18)
print("CREATOR ETH: %f" % CREATOR_ETH)
# All investors have the same share
INVESTOR_ETH = (coin_table["INVESTOR_1"]/(MARKET_TOTAL / 10**18)) * (RESERVE / 10**18)
print("BACKEND_REVENUE: %f" % (BACKEND_REVENUE / 10**18))
print("INVESTOR ETH: %f" % INVESTOR_ETH)
INVESTOR_PROFIT = INVESTOR_ETH - (INVESTMENT/10**18)
print("INVESTOR_PROFIT: %f" % INVESTOR_PROFIT)
MAKER_ETH = (coin_table["MAKER_1"]/(MARKET_TOTAL / 10**18)) * (RESERVE / 10**18)
print("MAKER ETH: %f" % MAKER_ETH)

Purchase 0
INVESTMENT_PRICE (ETH whole for MKT whole): 108
MINTED per maker: 0.000230
Purchase 1
INVESTMENT_PRICE (ETH whole for MKT whole): 113
MINTED per maker: 0.000220
Purchase 2
INVESTMENT_PRICE (ETH whole for MKT whole): 118
MINTED per maker: 0.000212
Purchase 3
INVESTMENT_PRICE (ETH whole for MKT whole): 122
MINTED per maker: 0.000203
Purchase 4
INVESTMENT_PRICE (ETH whole for MKT whole): 127
MINTED per maker: 0.000196
Purchase 5
INVESTMENT_PRICE (ETH whole for MKT whole): 132
MINTED per maker: 0.000189
Purchase 6
INVESTMENT_PRICE (ETH whole for MKT whole): 137
MINTED per maker: 0.000182
Purchase 7
INVESTMENT_PRICE (ETH whole for MKT whole): 141
MINTED per maker: 0.000176
Purchase 8
INVESTMENT_PRICE (ETH whole for MKT whole): 146
MINTED per maker: 0.000171
Purchase 9
INVESTMENT_PRICE (ETH whole for MKT whole): 151
MINTED per maker: 0.000165
Purchase 10
INVESTMENT_PRICE (ETH whole for MKT whole): 156
MINTED per maker: 0.000160
Purchase 11
INVESTMENT_PRICE (ETH whole for MKT whole